In [1]:
!python --version

# 3.12.10

Python 3.12.10


In [2]:
# !pip install -U rag-bench
# !pip install typing-extensions --upgrade
# !pip install razdel
# !pip install vllm
# !pip install -U \
#   "langchain<1.0" "langchain-core<1.0" "langchain-community<1.0" "langchain-text-splitters<1.0" \
#   "langchain-openai<1.0" "langchain-mistralai<1.0" "langchain-huggingface<1.0"

Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: langchain-core 0.3.81
Uninstalling langchain-core-0.3.81:
  Successfully uninstalled langchain-core-0.3.81
Found existing installation: langchain-community 0.3.31
Uninstalling langchain-community-0.3.31:
  Successfully uninstalled langchain-community-0.3.31
Found existing installation: langchain-text-splitters 0.3.11
Uninstalling langchain-text-splitters-0.3.11:
  Successfully uninstalled langchain-text-splitters-0.3.11
Found existing installation: langchain-openai 1.1.6
Uninstalling langchain-openai-1.1.6:
  Successfully uninstalled langchain-openai-1.1.6
Found existing installation: langchain-mistralai 1.1.1
Uninstalling langchain-mistralai-1.1.1:
  Successfully uninstalled langchain-mistralai-1.1.1
Found existing installation: langchain-huggingface 1.2.0
Uninstalling langchain-huggingface-1.2.0:
  Successfully uninstalled langchain-hug

In [27]:
import importlib
import os

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_openai import ChatOpenAI

from rag_bench import baseline, data, evaluator, results
from datasets import load_dataset


HIST_PRIVATE_QA_REPO_ID = "ai-forever/hist-rag-bench-private-qa"
HIST_PRIVATE_TEXTS_REPO_ID = "ai-forever/hist-rag-bench-private-texts"

def get_private_qa_dataset(version):
    return load_dataset(HIST_PRIVATE_QA_REPO_ID, revision=version)

def get_private_texts_dataset(version):
    return load_dataset(HIST_PRIVATE_TEXTS_REPO_ID, revision=version)

def get_public_to_private_texts_mapping(version):
    private_texts_ds = get_private_texts_dataset(version)

    mapping = {}
    for item in private_texts_ds["train"]:
        mapping[item["public_id"]] = item["id"]
    return mapping


#get public datasets (history ones)
texts_ds, questions_ds, version = data.get_datasets(is_hist=True)

#get private datasets (history ones)
qa_dataset = get_private_qa_dataset(version)
mapping = get_public_to_private_texts_mapping(version)


rag_evaluator = evaluator.RAGEvaluator()
mapping = get_public_to_private_texts_mapping(version)

# Latest texts version: 1.13.0
# Latest questions version: 1.13.0
# Loaded texts dataset with 560 texts
# Loaded questions dataset with 600 questions

Latest texts version: 1.13.0
Latest questions version: 1.13.0
Loaded texts dataset with 560 texts
Loaded questions dataset with 600 questions


[2025-12-29 14:25:15] INFO rouge_scorer.py:83: Using default tokenizer.
[2025-12-29 14:25:15] INFO rouge_scorer.py:83: Using default tokenizer.


In [5]:
# import os
# os.environ["VLLM_DISABLE_PROGRESS_BAR"] = "1"

from langchain_community.llms import VLLM


llm = VLLM(
    # model='google/gemma-3-4b-it',
    model='mistralai/Ministral-3-3B-Instruct-2512',
    trust_remote_code=True, 
    tensor_parallel_size=1,
    max_new_tokens=200,
    top_p=0.95,
    temperature=0.8,
)

INFO 12-29 12:47:43 [utils.py:253] non-default args: {'trust_remote_code': True, 'disable_log_stats': True, 'model': 'mistralai/Ministral-3-3B-Instruct-2512'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-29 12:48:07 [model.py:514] Resolved architecture: PixtralForConditionalGeneration
INFO 12-29 12:48:07 [model.py:1661] Using max model len 262144
INFO 12-29 12:48:07 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.


[2025-12-29 12:48:13] INFO tekken.py:195: Non special vocabulary size is 130072 with 1000 special tokens.


(EngineCore_DP0 pid=2923) INFO 12-29 12:48:14 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='mistralai/Ministral-3-3B-Instruct-2512', speculative_config=None, tokenizer='mistralai/Ministral-3-3B-Instruct-2512', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=262144, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None, kv_cache_metrics=F

...

(EngineCore_DP0 pid=2923) INFO 12-29 12:48:32 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')
(EngineCore_DP0 pid=2923) INFO 12-29 12:48:42 [weight_utils.py:487] Time spent downloading weights for mistralai/Ministral-3-3B-Instruct-2512: 9.370129 seconds
(EngineCore_DP0 pid=2923) INFO 12-29 12:48:42 [weight_utils.py:527] No consolidated.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:11<00:00, 11.19s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:11<00:00, 11.20s/it]
(EngineCore_DP0 pid=2923) 


(EngineCore_DP0 pid=2923) INFO 12-29 12:48:54 [default_loader.py:308] Loading weights took 11.41 seconds
(EngineCore_DP0 pid=2923) WARNING 12-29 12:48:54 [marlin_utils_fp8.py:99] Your GPU does not have native support for FP8 computation but FP8 quantization is being used. Weight-only FP8 compression will be used leveraging the Marlin kernel. This may degrade performance for compute-heavy workloads.
(EngineCore_DP0 pid=2923) INFO 12-29 12:48:55 [gpu_model_runner.py:3659] Model loading took 4.4367 GiB memory and 33.745246 seconds
(EngineCore_DP0 pid=2923) INFO 12-29 12:48:55 [gpu_model_runner.py:4446] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 2 image items of the maximum feature size.
(EngineCore_DP0 pid=2923) WARNING 12-29 12:48:55 [processing.py:1153] PixtralProcessorAdapter did not return `BatchFeature`. Make sure to match the behaviour of `ProcessorMixin` when implementing custom processors.
(EngineCore_DP0 pid=2923) INFO 12-29 12:49:08 [backen

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:05<00:00,  9.99it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:03<00:00, 10.27it/s]


(EngineCore_DP0 pid=2923) INFO 12-29 12:49:35 [gpu_model_runner.py:4587] Graph capturing finished in 9 secs, took 0.48 GiB
(EngineCore_DP0 pid=2923) INFO 12-29 12:49:35 [core.py:259] init engine (profile, create kv cache, warmup model) took 40.20 seconds
INFO 12-29 12:49:36 [llm.py:360] Supported tasks: ['generate']


In [6]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased"
)

llm_prompt = """Answer a question using the provided context. Return answer only. Respond in Russian.

<context>
{context}
</context>

Question: {input}"""

retrieval = baseline.init_retriever(
    texts_ds,
    embedding_model,
    top_k=5,
    chunk_size=1000,
    chunk_overlap=100,
)
generation = baseline.init_generation(retrieval, llm, llm_prompt=llm_prompt)

[2025-12-29 12:49:42] INFO SentenceTransformer.py:211: Use pytorch device_name: cuda:0
[2025-12-29 12:49:42] INFO SentenceTransformer.py:219: Load pretrained SentenceTransformer: sentence-transformers/distiluse-base-multilingual-cased
[2025-12-29 12:50:26] INFO posthog.py:22: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Initializing retriever
> Creating vector store
> Vector store created for 560 documents
> Done for 2.75 seconds
Initializing generation chain
> Done


In [7]:
res = baseline.get_results(
    generation, questions_ds, write_logs=False
)

Calculating and preparing results


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]/home/jovyan/.mlspace/envs/rag_1/lib/python3.12/site-packages/mistral_common/tokens/tokenizers/tekken.py:462: FutureWarning: `get_control_token` is deprecated. Use `get_special_token` instead.
  warnings.warn("`get_control_token` is deprecated. Use `get_special_token` instead.", FutureWarning)
Adding requests: 100%|██████████| 1/1 [00:00<00:00, 173.85it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]/home/jovyan/.mlspace/envs/rag_1/lib/python3.12/site-packages/mistral_common/tokens/tokenizers/tekken.py:462: FutureWarning: `get_control_token` is deprecated. Use `get_special_token` instead.
  warnings.warn("`get_control_token` is deprecated. Use `get_special_token` instead.", FutureWarning)
Adding requests: 100%|██████████| 1/1 [00:00<00:00, 431.25it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]/home/jovyan/.mlspace/envs/rag_1/lib/python3.12/site-packages/mistral_common/tokens/tokenizers/tekken.py:462: FutureWa

Done for 194.44 seconds


In [24]:
#saving the baseline results

res_path = "./test.json"
results.save(res, res_path)
res = results.load(res_path)

In [26]:
res['0']

{'found_ids': [183, 362, 178, 9, 162],
 'model_answer': ' В начале октября?\n\nВ тексте не упоминается конкретное атмосферное давление, лишь говорится, что оно усиливает перепады температур и связано с холодными потоками воздуха.'}

In [11]:
version

'1.13.0'

In [13]:
#calculate metrics

evaluation_results = evaluator.evaluate_rag_results(res, qa_dataset, mapping, rag_evaluator)

[2025-12-29 12:59:19] INFO rouge_scorer.py:83: Using default tokenizer.
[2025-12-29 12:59:19] INFO rouge_scorer.py:83: Using default tokenizer.


In [43]:
#average metrics

res = evaluation_results['average_metrics']

df_retrieval = pd.DataFrame([res["retrieval"]], index=["retrieval"]).apply(pd.to_numeric, errors="coerce")
df_generation = pd.DataFrame([res["generation"]], index=["generation"]).apply(pd.to_numeric, errors="coerce")

df_retrieval = df_retrieval.round(4)
df_generation = df_generation.round(4)

In [44]:
df_retrieval

,hit_rate,mrr,precision
retrieval,0.5017,0.3492,0.105


In [45]:
df_generation

,rouge1,rougeL,substr_match
generation,0.0513,0.0474,0.1233


In [53]:
#metrics by category

res_agg = evaluation_results['aggregated_metrics']

df = (
    pd.json_normalize(res_agg, sep="_")
      .T                               
      .rename(columns=lambda c: c)
)
df[0] = df[0].astype(float) if 0 in df.columns else df.iloc[:, 0].astype(float)

df = pd.DataFrame({k: pd.json_normalize(v, sep="_").iloc[0] for k, v in res_agg.items()}).T
df = df.apply(pd.to_numeric)
df = df.T
df = df.reindex(columns=["simple", "cond", "set", "mh", "overall"])

row_order = [
    "retrieval_hit_rate",
    "retrieval_mrr",
    "retrieval_precision",
    "generation_rouge1",
    "generation_rougeL",
    "generation_substr_match",
]
df = df.reindex(row_order)

df = df.round(4)

df

,simple,cond,set,mh,overall
retrieval_hit_rate,0.4400,0.5600,0.4600,0.5467,0.5017
retrieval_mrr,0.2831,0.4156,0.3274,0.3708,0.3492
retrieval_precision,0.0947,0.1147,0.0987,0.1120,0.1050
generation_rouge1,0.0413,0.0501,0.0710,0.0427,0.0513
generation_rougeL,0.0413,0.0501,0.0565,0.0418,0.0474
generation_substr_match,0.1533,0.2000,0.0000,0.1400,0.1233
